In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
dataset_path = "./drive/MyDrive/seed"

# start

# Transfer Learning to extract feature vector and rearranging those feature vectors into 8*9 map

In [ ]:
# we have already extrcated features with transfer learning and are saved in file extract_TL_features_inception_v3.py, 
# we will use those fetures which are already arranged in 8x9 map as an image to a cnn model.

# CNN

# kfold 5, batch size 32

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_path = "./drive/MyDrive/seed"

In [ ]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.layers import Input, Conv2D, MaxPooling2D, Conv3D, MaxPooling3D, Dropout
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM
from keras.models import Sequential, Model

from keras.regularizers import l2

from sklearn.metrics import confusion_matrix,  classification_report, f1_score, precision_score, recall_score

import keras
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from keras import backend as K
import time
from sklearn.model_selection import StratifiedKFold

import seaborn as sns

In [ ]:
num_classes = 3
batch_size = 64
img_rows, img_cols, fft_length, num_chan = 8, 9, 2048, 1

In [ ]:
i = 1

# X = np.load(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/{i}_{i}_X_1D.npy")
y = np.load(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/{i}_{i}_y.npy")


In [ ]:
y.shape

(987,)

In [ ]:
# one_y = np.array([y[:1697]] * 3).reshape((-1,))
# one_y = to_categorical(one_y, num_classes)
one_y_1 = np.array([y[:]]*3).reshape((-1,))
print(one_y_1.shape)
# for i in range(3378):
#   if one_y_1[i] != y[(i%1126)]:
#     print("break")
#     break
print(one_y_1[0:10])
print(np.unique(one_y_1, return_counts= True))
one_y_1 = to_categorical(one_y_1, num_classes)
print(one_y_1[0:10])


(2961,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
(array([-1.,  0.,  1.]), array([ 978,  963, 1020]))
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [ ]:
print(np.unique(one_y_1.argmax(1), return_counts= True))

(array([0, 1, 2]), array([ 963, 1020,  978]))


In [ ]:
import gc

In [ ]:
acc_list = []
std_list = []
all_acc = []
acc_log = {}
flag_for_model_summary = 0
# with tf.device('/device:GPU:0'):
for subject in range(1,16):  # 15 models for 15 subjects, no inter subject training, only intra subject
    K.clear_session()
    start = time.time()
    falx = np.empty((0, 8, 9, 2048))
    gc.collect()
    for exp_id in range(1,4):
      temp = np.load(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/{subject}_{exp_id}_X89.npy")
      falx = np.vstack([falx, temp])
      temp = None

#     one_falx_1 = falx#.reshape(1008, 8, 9, 1280)

    # ###============= random select ============####
    # permutation = np.random.permutation(one_y_1.shape[0])
    # one_falx_2 = one_falx_1[permutation, :]
    # one_falx = one_falx_2[0:3400]
    # one_y_2 = one_y_1[permutation, :]
    # one_y = one_y_2[0:3400]
    # ###============= random select ============####
    one_y = one_y_1
#     one_falx = one_falx_1[:,:,:,:]

    print(one_y.shape)
    print(falx.shape)
    # x_train, x_test, y_train, y_test = train_test_split(one_falx, one_y, test_size=0.25)
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []


    # create model
    for k,(train, test) in enumerate(kfold.split(falx, one_y.argmax(1))):
        img_size = (img_rows, img_cols, 2048)


        def create_base_network(input_dim):

            seq = Sequential()
            seq.add(Conv2D(16, 3, activation='relu', padding='same', name='conv1',
                           input_shape=input_dim))#, kernel_regularizer=keras.regularizers.l2(0.001)))
            seq.add(Conv2D(32, 2, activation='relu', padding='same', name='conv2'))#, kernel_regularizer=keras.regularizers.l2(0.001)))
            # seq.add(Conv3D(32, 3, activation='relu', padding='same', name='conv3'))
            seq.add(Conv2D(8, 1, activation='relu', padding='same', name='conv4'))
            seq.add(MaxPooling2D(2, 2, name='pool1'))
            seq.add(Flatten(name='fla1'))
            seq.add(Dense(16, activation='relu', name='dense1'))#, kernel_regularizer=keras.regularizers.l2(0.01)))
#             seq.add(Reshape((1, 64), name='reshape'))

            return seq




        base_network = create_base_network(img_size)
        input_1 = Input(shape=img_size)
#         input_2 = Input(shape=img_size)
#         input_3 = Input(shape=img_size)
#         input_4 = Input(shape=img_size)
#         input_5 = Input(shape=img_size)
#         input_6 = Input(shape=img_size)



#         out_all = Concatenate(axis=1)(base_network(input_1))
#         lstm_layer = LSTM(32, name='lstm')(out_all)
        out_layer = Dense(3, activation='softmax', name='out')(base_network(input_1))
        model = Model(input_1, out_layer)
        # model.summary()
        if flag_for_model_summary==0:
          print(model.summary())
          flag_for_model_summary = 1

        # Compile model
        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=['accuracy'])
        # Fit the model
        x_train = falx[train]
        y_train = one_y[train]

        x_test = falx[test]
        y_test = one_y[test]

        gc.collect()

        history = model.fit(x_train, y_train,
                            validation_data=(x_test, y_test),
                            epochs=150, batch_size=32, verbose=0,
                            callbacks=[
                                          tf.keras.callbacks.EarlyStopping(
                                              monitor='val_accuracy',
                                              patience=30,
                                              restore_best_weights=True
                                          )
                                      ])

        plt.plot(history.history['accuracy'], label='acc (training data)')
        plt.plot(history.history['val_accuracy'], label='acc (validation data)')
        plt.plot(history.history['loss'], label='loss (training data)')
        plt.plot(history.history['val_loss'], label='loss (validation data)')
        plt.title('training visualisation')
        plt.ylabel('Value')
        plt.xlabel('No. epoch')
        plt.legend()
        plt.tight_layout()
        plt.show()

        # evaluate the model

        scores = model.evaluate(x_test, y_test, verbose=0)

        print("%.2f%%" % (scores[1] * 100)) # Accuracy
        all_acc.append(scores[1] * 100)

        y_pred = np.array(list(map(lambda x: np.argmax(x), model.predict(x_test))))
        cm = confusion_matrix(y_test.argmax(1), y_pred)
        clr = classification_report(y_test.argmax(1), y_pred, target_names=(["0","1","2"]))
        plt.figure(figsize=(7, 7))
        sns.heatmap(cm, annot=True, vmin=0, fmt='g', cbar=False, cmap='Blues')
        plt.xticks(np.arange(3) + 0.5, (["0","1","2"]))
        plt.yticks(np.arange(3) + 0.5, (["0","1","2"]))
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Confusion Matrix")
        plt.show()
        print("Classification Report:\n----------------------\n", clr)

        print(f"subject_no.:{subject}'s kfold:{k+1} completed")



    # print("all acc: {}".format(all_acc))
    print("mean acc: {}".format(np.mean(all_acc)))
    print("std acc: {}".format(np.std(all_acc)))
    acc_list.append(np.mean(all_acc))
    std_list.append(np.std(all_acc))
    print("subject_no.： {}".format(subject))
    acc_log[subject] = all_acc
    all_acc = []
    end = time.time()
    print("%.2f" % (end - start))   # run time

In [ ]:
gc.collect()

In [ ]:
print('Acc_all: {}'.format(acc_list))
print('Std_all: {}'.format(std_list))
print("Acc_mean: {}".format(np.mean(acc_list)))
print("Std_all: {}".format(np.std(std_list)))

In [ ]:
print(acc_log)

In [ ]:
np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/acc_list_k5.npy", acc_list)
np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/std_list_k5.npy", std_list)
np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/acc_log_k5.npy", acc_log)

# kfold 5 batch 128

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_path = "./drive/MyDrive/seed"

In [ ]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.layers import Input, Conv2D, MaxPooling2D, Conv3D, MaxPooling3D, Dropout
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM
from keras.models import Sequential, Model

from keras.regularizers import l2

from sklearn.metrics import confusion_matrix,  classification_report, f1_score, precision_score, recall_score

import keras
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from keras import backend as K
import time
from sklearn.model_selection import StratifiedKFold

import seaborn as sns

In [ ]:
num_classes = 3
batch_size = 64
img_rows, img_cols, fft_length, num_chan = 8, 9, 2048, 1

In [ ]:
i = 1

# X = np.load(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/{i}_{i}_X_1D.npy")
y = np.load(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/{i}_{i}_y.npy")


In [ ]:
y.shape

(987,)

In [ ]:
# one_y = np.array([y[:1697]] * 3).reshape((-1,))
# one_y = to_categorical(one_y, num_classes)
one_y_1 = np.array([y[:]]*3).reshape((-1,))
print(one_y_1.shape)
# for i in range(3378):
#   if one_y_1[i] != y[(i%1126)]:
#     print("break")
#     break
print(one_y_1[0:10])
print(np.unique(one_y_1, return_counts= True))
one_y_1 = to_categorical(one_y_1, num_classes)
print(one_y_1[0:10])


(2961,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
(array([-1.,  0.,  1.]), array([ 978,  963, 1020]))
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [ ]:
print(np.unique(one_y_1.argmax(1), return_counts= True))

(array([0, 1, 2]), array([ 963, 1020,  978]))


In [ ]:
import gc

In [ ]:
acc_list = []
std_list = []
all_acc = []
acc_log = {}
flag_for_model_summary = 0
# with tf.device('/device:GPU:0'):
for subject in range(1,16):  # 15 models for 15 subjects, no inter subject training, only intra subject
    K.clear_session()
    start = time.time()
    falx = np.empty((0, 8, 9, 2048))
    for exp_id in range(1,4):
      temp = np.load(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/{subject}_{exp_id}_X89.npy")
      falx = np.vstack([falx, temp])
      temp = None

#     one_falx_1 = falx#.reshape(1008, 8, 9, 1280)

    # ###============= random select ============####
    # permutation = np.random.permutation(one_y_1.shape[0])
    # one_falx_2 = one_falx_1[permutation, :]
    # one_falx = one_falx_2[0:3400]
    # one_y_2 = one_y_1[permutation, :]
    # one_y = one_y_2[0:3400]
    # ###============= random select ============####
    one_y = one_y_1
#     one_falx = one_falx_1[:,:,:,:]

    print(one_y.shape)
    print(falx.shape)
    # x_train, x_test, y_train, y_test = train_test_split(one_falx, one_y, test_size=0.25)
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cvscores = []


    # create model
    for k,(train, test) in enumerate(kfold.split(falx, one_y.argmax(1))):
        img_size = (img_rows, img_cols, 2048)


        def create_base_network(input_dim):

            seq = Sequential()
            seq.add(Conv2D(16, 3, activation='relu', padding='same', name='conv1',
                           input_shape=input_dim))#, kernel_regularizer=keras.regularizers.l2(0.001)))
            seq.add(Conv2D(32, 2, activation='relu', padding='same', name='conv2'))#, kernel_regularizer=keras.regularizers.l2(0.001)))
            # seq.add(Conv3D(32, 3, activation='relu', padding='same', name='conv3'))
            seq.add(Conv2D(8, 1, activation='relu', padding='same', name='conv4'))
            seq.add(MaxPooling2D(2, 2, name='pool1'))
            seq.add(Flatten(name='fla1'))
            seq.add(Dense(16, activation='relu', name='dense1'))#, kernel_regularizer=keras.regularizers.l2(0.01)))
#             seq.add(Reshape((1, 64), name='reshape'))

            return seq




        base_network = create_base_network(img_size)
        input_1 = Input(shape=img_size)
#         input_2 = Input(shape=img_size)
#         input_3 = Input(shape=img_size)
#         input_4 = Input(shape=img_size)
#         input_5 = Input(shape=img_size)
#         input_6 = Input(shape=img_size)



#         out_all = Concatenate(axis=1)(base_network(input_1))
#         lstm_layer = LSTM(32, name='lstm')(out_all)
        out_layer = Dense(3, activation='softmax', name='out')(base_network(input_1))
        model = Model(input_1, out_layer)
        # model.summary()
        if flag_for_model_summary==0:
          print(model.summary())
          flag_for_model_summary = 1

        # Compile model
        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=['accuracy'])
        # Fit the model
        x_train = falx[train]
        y_train = one_y[train]

        x_test = falx[test]
        y_test = one_y[test]

        gc.collect()

        history = model.fit(x_train, y_train,
                            validation_data=(x_test, y_test),
                            epochs=150, batch_size=128, verbose=0,
                            callbacks=[
                                          tf.keras.callbacks.EarlyStopping(
                                              monitor='val_accuracy',
                                              patience=30,
                                              restore_best_weights=True
                                          )
                                      ])

        plt.plot(history.history['accuracy'], label='acc (training data)')
        plt.plot(history.history['val_accuracy'], label='acc (validation data)')
        plt.plot(history.history['loss'], label='loss (training data)')
        plt.plot(history.history['val_loss'], label='loss (validation data)')
        plt.title('training visualisation')
        plt.ylabel('Value')
        plt.xlabel('No. epoch')
        plt.legend()
        plt.tight_layout()
        plt.show()

        # evaluate the model

        scores = model.evaluate(x_test, y_test, verbose=0)

        print("%.2f%%" % (scores[1] * 100)) # Accuracy
        all_acc.append(scores[1] * 100)

        y_pred = np.array(list(map(lambda x: np.argmax(x), model.predict(x_test))))
        cm = confusion_matrix(y_test.argmax(1), y_pred)
        clr = classification_report(y_test.argmax(1), y_pred, target_names=(["0","1","2"]))
        plt.figure(figsize=(7, 7))
        sns.heatmap(cm, annot=True, vmin=0, fmt='g', cbar=False, cmap='Blues')
        plt.xticks(np.arange(3) + 0.5, (["0","1","2"]))
        plt.yticks(np.arange(3) + 0.5, (["0","1","2"]))
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Confusion Matrix")
        plt.show()
        print("Classification Report:\n----------------------\n", clr)

        print(f"subject_no.:{subject}'s kfold:{k+1} completed")



    # print("all acc: {}".format(all_acc))
    print("mean acc: {}".format(np.mean(all_acc)))
    print("std acc: {}".format(np.std(all_acc)))
    acc_list.append(np.mean(all_acc))
    std_list.append(np.std(all_acc))
    print("subject_no.： {}".format(subject))
    acc_log[subject] = all_acc
    all_acc = []
    end = time.time()
    print("%.2f" % (end - start))   # run time

In [ ]:
gc.collect()

In [ ]:
print('Acc_all: {}'.format(acc_list))
print('Std_all: {}'.format(std_list))
print("Acc_mean: {}".format(np.mean(acc_list)))
print("Std_all: {}".format(np.std(std_list)))

In [ ]:
print(acc_log)

In [ ]:
np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/acc_list_k5_128.npy", acc_list)
np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/std_list_k5_128.npy", std_list)
np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/acc_log_k5_128.npy", acc_log)

# kfold 10, batch size 32



In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Apr 23 14:34:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!nvidia-smi

Wed Apr 20 07:01:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available 1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# %reset -f

In [ ]:
dataset_path = "./drive/MyDrive/seed"

In [ ]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.layers import Input, Conv2D, MaxPooling2D, Conv3D, MaxPooling3D, Dropout
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM
from keras.models import Sequential, Model

from keras.regularizers import l2

from sklearn.metrics import confusion_matrix,  classification_report, f1_score, precision_score, recall_score

import keras
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from keras import backend as K
import time
from sklearn.model_selection import StratifiedKFold

import seaborn as sns

In [ ]:
num_classes = 3
batch_size = 64
img_rows, img_cols, fft_length, num_chan = 8, 9, 2048, 1

In [ ]:
i = 1

# X = np.load(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/{i}_{i}_X_1D.npy")
y = np.load(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/{i}_{i}_y.npy")


In [ ]:
# subject = 2
# exp_id = 3

# x89 = np.load(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_mobilenet_v2/{subject}_{exp_id}_X89.npy")

In [ ]:
# x89.shape

In [ ]:
y.shape

(987,)

In [ ]:
# one_y = np.array([y[:1697]] * 3).reshape((-1,))
# one_y = to_categorical(one_y, num_classes)
one_y_1 = np.array([y[:]]*3).reshape((-1,))
print(one_y_1.shape)
# for i in range(3378):
#   if one_y_1[i] != y[(i%1126)]:
#     print("break")
#     break
print(one_y_1[0:10])
print(np.unique(one_y_1, return_counts= True))
one_y_1 = to_categorical(one_y_1, num_classes)
print(one_y_1[0:10])


(2961,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
(array([-1.,  0.,  1.]), array([ 978,  963, 1020]))
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [ ]:
print(np.unique(one_y_1.argmax(1), return_counts= True))

(array([0, 1, 2]), array([ 963, 1020,  978]))


In [ ]:
import gc

In [ ]:
acc_list = []
std_list = []
all_acc = []
acc_log = {}
flag_for_model_summary = 0
# with tf.device('/device:GPU:0'):
for subject in range(1,16):  # 15 models for 15 subjects, no inter subject training, only intra subject
    K.clear_session()
    start = time.time()
    falx = np.empty((0, 8, 9, 2048))
    for exp_id in range(1,4):
      temp = np.load(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/{subject}_{exp_id}_X89.npy")
      falx = np.vstack([falx, temp])
      temp = None

#     one_falx_1 = falx#.reshape(1008, 8, 9, 1280)

    # ###============= random select ============####
    # permutation = np.random.permutation(one_y_1.shape[0])
    # one_falx_2 = one_falx_1[permutation, :]
    # one_falx = one_falx_2[0:3400]
    # one_y_2 = one_y_1[permutation, :]
    # one_y = one_y_2[0:3400]
    # ###============= random select ============####
    one_y = one_y_1
#     one_falx = one_falx_1[:,:,:,:]

    print(one_y.shape)
    print(falx.shape)
    # x_train, x_test, y_train, y_test = train_test_split(one_falx, one_y, test_size=0.25)
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    cvscores = []


    # create model
    for k,(train, test) in enumerate(kfold.split(falx, one_y.argmax(1))):
        img_size = (img_rows, img_cols, 2048)


        def create_base_network(input_dim):

            seq = Sequential()
            seq.add(Conv2D(16, 3, activation='relu', padding='same', name='conv1',
                           input_shape=input_dim))#, kernel_regularizer=keras.regularizers.l2(0.001)))
            seq.add(Conv2D(32, 2, activation='relu', padding='same', name='conv2'))#, kernel_regularizer=keras.regularizers.l2(0.001)))
            # seq.add(Conv3D(32, 3, activation='relu', padding='same', name='conv3'))
            seq.add(Conv2D(8, 1, activation='relu', padding='same', name='conv4'))
            seq.add(MaxPooling2D(2, 2, name='pool1'))
            seq.add(Flatten(name='fla1'))
            seq.add(Dense(16, activation='relu', name='dense1'))#, kernel_regularizer=keras.regularizers.l2(0.01)))
#             seq.add(Reshape((1, 64), name='reshape'))

            return seq




        base_network = create_base_network(img_size)
        input_1 = Input(shape=img_size)
#         input_2 = Input(shape=img_size)
#         input_3 = Input(shape=img_size)
#         input_4 = Input(shape=img_size)
#         input_5 = Input(shape=img_size)
#         input_6 = Input(shape=img_size)



#         out_all = Concatenate(axis=1)(base_network(input_1))
#         lstm_layer = LSTM(32, name='lstm')(out_all)
        out_layer = Dense(3, activation='softmax', name='out')(base_network(input_1))
        model = Model(input_1, out_layer)
        # model.summary()
        if flag_for_model_summary==0:
          print(model.summary())
          flag_for_model_summary = 1

        # Compile model
        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=['accuracy'])
        # Fit the model
        x_train = falx[train]
        y_train = one_y[train]

        x_test = falx[test]
        y_test = one_y[test]

        gc.collect()

        history = model.fit(x_train, y_train,
                            validation_data=(x_test, y_test),
                            epochs=150, batch_size=32, verbose=0,
                            callbacks=[
                                          tf.keras.callbacks.EarlyStopping(
                                              monitor='val_accuracy',
                                              patience=30,
                                              restore_best_weights=True
                                          )
                                      ])

        plt.plot(history.history['accuracy'], label='acc (training data)')
        plt.plot(history.history['val_accuracy'], label='acc (validation data)')
        plt.plot(history.history['loss'], label='loss (training data)')
        plt.plot(history.history['val_loss'], label='loss (validation data)')
        plt.title('training visualisation')
        plt.ylabel('Value')
        plt.xlabel('No. epoch')
        plt.legend()
        plt.tight_layout()
        plt.show()

        # evaluate the model

        scores = model.evaluate(x_test, y_test, verbose=0)

        print("%.2f%%" % (scores[1] * 100)) # Accuracy
        all_acc.append(scores[1] * 100)

        y_pred = np.array(list(map(lambda x: np.argmax(x), model.predict(x_test))))
        cm = confusion_matrix(y_test.argmax(1), y_pred)
        clr = classification_report(y_test.argmax(1), y_pred, target_names=(["0","1","2"]))
        plt.figure(figsize=(7, 7))
        sns.heatmap(cm, annot=True, vmin=0, fmt='g', cbar=False, cmap='Blues')
        plt.xticks(np.arange(3) + 0.5, (["0","1","2"]))
        plt.yticks(np.arange(3) + 0.5, (["0","1","2"]))
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Confusion Matrix")
        plt.show()
        print("Classification Report:\n----------------------\n", clr)

        print(f"subject_no.:{subject}'s kfold:{k+1} completed")



    # print("all acc: {}".format(all_acc))
    print("mean acc: {}".format(np.mean(all_acc)))
    print("std acc: {}".format(np.std(all_acc)))
    acc_list.append(np.mean(all_acc))
    std_list.append(np.std(all_acc))
    print("subject_no.： {}".format(subject))
    acc_log[subject] = all_acc
    all_acc = []
    end = time.time()
    print("%.2f" % (end - start))   # run time

In [ ]:
gc.collect()

In [ ]:
print('Acc_all: {}'.format(acc_list))
print('Std_all: {}'.format(std_list))
print("Acc_mean: {}".format(np.mean(acc_list)))
print("Std_all: {}".format(np.std(std_list)))

In [ ]:
print(acc_log)

In [ ]:
np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/acc_list.npy", acc_list)
np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/std_list.npy", std_list)
np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/acc_log.npy", acc_log)

# kfold 10 batch 128

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset_path = "./drive/MyDrive/seed"

In [ ]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.layers import Input, Conv2D, MaxPooling2D, Conv3D, MaxPooling3D, Dropout
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM
from keras.models import Sequential, Model

from keras.regularizers import l2

from sklearn.metrics import confusion_matrix,  classification_report, f1_score, precision_score, recall_score

import keras
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from keras import backend as K
import time
from sklearn.model_selection import StratifiedKFold

import seaborn as sns

In [ ]:
num_classes = 3
batch_size = 64
img_rows, img_cols, fft_length, num_chan = 8, 9, 2048, 1

In [ ]:
i = 1

# X = np.load(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/{i}_{i}_X_1D.npy")
y = np.load(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/{i}_{i}_y.npy")


In [ ]:
y.shape

(987,)

In [ ]:
# one_y = np.array([y[:1697]] * 3).reshape((-1,))
# one_y = to_categorical(one_y, num_classes)
one_y_1 = np.array([y[:]]*3).reshape((-1,))
print(one_y_1.shape)
# for i in range(3378):
#   if one_y_1[i] != y[(i%1126)]:
#     print("break")
#     break
print(one_y_1[0:10])
print(np.unique(one_y_1, return_counts= True))
one_y_1 = to_categorical(one_y_1, num_classes)
print(one_y_1[0:10])


(2961,)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
(array([-1.,  0.,  1.]), array([ 978,  963, 1020]))
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [ ]:
print(np.unique(one_y_1.argmax(1), return_counts= True))

(array([0, 1, 2]), array([ 963, 1020,  978]))


In [ ]:
import gc

In [ ]:
acc_list = []
std_list = []
all_acc = []
acc_log = {}
flag_for_model_summary = 0
# with tf.device('/device:GPU:0'):
for subject in range(1,16):  # 15 models for 15 subjects, no inter subject training, only intra subject
    K.clear_session()
    start = time.time()
    falx = np.empty((0, 8, 9, 2048))
    for exp_id in range(1,4):
      temp = np.load(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/{subject}_{exp_id}_X89.npy")
      falx = np.vstack([falx, temp])
      temp = None

#     one_falx_1 = falx#.reshape(1008, 8, 9, 1280)

    # ###============= random select ============####
    # permutation = np.random.permutation(one_y_1.shape[0])
    # one_falx_2 = one_falx_1[permutation, :]
    # one_falx = one_falx_2[0:3400]
    # one_y_2 = one_y_1[permutation, :]
    # one_y = one_y_2[0:3400]
    # ###============= random select ============####
    one_y = one_y_1
#     one_falx = one_falx_1[:,:,:,:]

    print(one_y.shape)
    print(falx.shape)
    # x_train, x_test, y_train, y_test = train_test_split(one_falx, one_y, test_size=0.25)
    seed = 7
    np.random.seed(seed)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    cvscores = []


    # create model
    for k,(train, test) in enumerate(kfold.split(falx, one_y.argmax(1))):
        img_size = (img_rows, img_cols, 2048)


        def create_base_network(input_dim):

            seq = Sequential()
            seq.add(Conv2D(16, 3, activation='relu', padding='same', name='conv1',
                           input_shape=input_dim))#, kernel_regularizer=keras.regularizers.l2(0.001)))
            seq.add(Conv2D(32, 2, activation='relu', padding='same', name='conv2'))#, kernel_regularizer=keras.regularizers.l2(0.001)))
            # seq.add(Conv3D(32, 3, activation='relu', padding='same', name='conv3'))
            seq.add(Conv2D(8, 1, activation='relu', padding='same', name='conv4'))
            seq.add(MaxPooling2D(2, 2, name='pool1'))
            seq.add(Flatten(name='fla1'))
            seq.add(Dense(16, activation='relu', name='dense1'))#, kernel_regularizer=keras.regularizers.l2(0.01)))
#             seq.add(Reshape((1, 64), name='reshape'))

            return seq




        base_network = create_base_network(img_size)
        input_1 = Input(shape=img_size)
#         input_2 = Input(shape=img_size)
#         input_3 = Input(shape=img_size)
#         input_4 = Input(shape=img_size)
#         input_5 = Input(shape=img_size)
#         input_6 = Input(shape=img_size)



#         out_all = Concatenate(axis=1)(base_network(input_1))
#         lstm_layer = LSTM(32, name='lstm')(out_all)
        out_layer = Dense(3, activation='softmax', name='out')(base_network(input_1))
        model = Model(input_1, out_layer)
        # model.summary()
        if flag_for_model_summary==0:
          print(model.summary())
          flag_for_model_summary = 1

        # Compile model
        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=['accuracy'])
        # Fit the model
        x_train = falx[train]
        y_train = one_y[train]

        x_test = falx[test]
        y_test = one_y[test]

        gc.collect()

        history = model.fit(x_train, y_train,
                            validation_data=(x_test, y_test),
                            epochs=150, batch_size=128, verbose=0,
                            callbacks=[
                                          tf.keras.callbacks.EarlyStopping(
                                              monitor='val_accuracy',
                                              patience=30,
                                              restore_best_weights=True
                                          )
                                      ])

        plt.plot(history.history['accuracy'], label='acc (training data)')
        plt.plot(history.history['val_accuracy'], label='acc (validation data)')
        plt.plot(history.history['loss'], label='loss (training data)')
        plt.plot(history.history['val_loss'], label='loss (validation data)')
        plt.title('training visualisation')
        plt.ylabel('Value')
        plt.xlabel('No. epoch')
        plt.legend()
        plt.tight_layout()
        plt.show()

        # evaluate the model

        scores = model.evaluate(x_test, y_test, verbose=0)

        print("%.2f%%" % (scores[1] * 100)) # Accuracy
        all_acc.append(scores[1] * 100)

        y_pred = np.array(list(map(lambda x: np.argmax(x), model.predict(x_test))))
        cm = confusion_matrix(y_test.argmax(1), y_pred)
        clr = classification_report(y_test.argmax(1), y_pred, target_names=(["0","1","2"]))
        plt.figure(figsize=(7, 7))
        sns.heatmap(cm, annot=True, vmin=0, fmt='g', cbar=False, cmap='Blues')
        plt.xticks(np.arange(3) + 0.5, (["0","1","2"]))
        plt.yticks(np.arange(3) + 0.5, (["0","1","2"]))
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Confusion Matrix")
        plt.show()
        print("Classification Report:\n----------------------\n", clr)

        print(f"subject_no.:{subject}'s kfold:{k+1} completed")



    # print("all acc: {}".format(all_acc))
    print("mean acc: {}".format(np.mean(all_acc)))
    print("std acc: {}".format(np.std(all_acc)))
    acc_list.append(np.mean(all_acc))
    std_list.append(np.std(all_acc))
    print("subject_no.： {}".format(subject))
    acc_log[subject] = all_acc
    all_acc = []
    end = time.time()
    print("%.2f" % (end - start))   # run time

In [ ]:
gc.collect()

In [ ]:
print('Acc_all: {}'.format(acc_list))
print('Std_all: {}'.format(std_list))
print("Acc_mean: {}".format(np.mean(acc_list)))
print("Std_all: {}".format(np.std(std_list)))

In [ ]:
print(acc_log)

In [ ]:
np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/acc_list_k10_128.npy", acc_list)
np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/std_list_k10_128.npy", std_list)
np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/acc_log_k10_128.npy", acc_log)